In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
df = pd.read_csv("Data/autos.csv",header=0, sep=',',encoding='latin1',)

In [3]:
# print(df.seller.value_counts())
# df[df.seller != 'gewerblich']
df=df.drop('seller',1)

# print(df.offerType.value_counts())
# df[df.offerType != 'Gesuch']
df=df.drop('offerType',1)

<ipython-input-3-56df28571f9b>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop('seller',1)
<ipython-input-3-56df28571f9b>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop('offerType',1)


In [4]:
print(df.shape)
df = df[(df.powerPS > 50) & (df.powerPS < 900)]
print(df.shape)

df = df[(df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)]
print(df.shape)

(371824, 18)
(319957, 18)
(309412, 18)


In [5]:
df.drop(['name','abtest','dateCrawled','nrOfPictures','lastSeen','postalCode','dateCreated'], axis='columns', inplace=True)

In [6]:
new_df = df.copy()
new_df = new_df.drop_duplicates(['price','vehicleType','yearOfRegistration','gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType'
,'notRepairedDamage'])

In [7]:
new_df.gearbox.replace(('manuell','automatik'),('manual','automatic'),inplace=True)
new_df.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electric'),inplace=True)
new_df.vehicleType.replace(('kleinwagen','cabrio','kombi','andere'),
                            ('small car','convertible','combination','others'),inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','NO'),inplace=True)

In [8]:
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 15000)]

new_df['notRepairedDamage'].fillna(value='not-declared', inplace=True)
new_df['fuelType'].fillna(value='not-declared', inplace=True)
new_df['gearbox'].fillna(value='not-declared', inplace=True)
new_df['vehicleType'].fillna(value='not-declared', inplace=True)
new_df['model'].fillna(value='not-declared', inplace=True)

In [9]:
new_df.to_csv("Data/autos_preprocessed.csv")
df.shape

(309412, 11)

In [10]:
new_df = pd.read_csv("Data/autos_preprocessed.csv",header=0, sep=',',encoding='latin1',)

<b>Cleaning The Dataset</b>

In [11]:
labels = ['gearbox', 'notRepairedDamage','model','brand','fuelType','vehicleType']

mapper = {}
for i in labels: 
    mapper[i] = LabelEncoder()
    mapper[i].fit(new_df[i])
    tr = mapper[i].transform(new_df[i])
    np.save('Data/'+str('classes'+i+'.npy'), mapper[i].classes_)
    print(i,";",mapper[i])
    new_df.loc[:,i+'_labels'] = pd.Series(tr, index=new_df.index)

labeled = new_df[['price'
                        ,'yearOfRegistration'
                        ,'powerPS'
                        ,'kilometer'
                        ,'monthOfRegistration'
                        ]
                    + [x+"_labels" for x in labels]]

print(labeled.columns)

gearbox ; LabelEncoder()
notRepairedDamage ; LabelEncoder()
model ; LabelEncoder()
brand ; LabelEncoder()
fuelType ; LabelEncoder()
vehicleType ; LabelEncoder()
Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


<b>Splitting Data into Independant And Dependant Variables</b>

In [12]:
Y = labeled.iloc[:,0].values
X = labeled.iloc[:,1:].values

Y = Y.reshape(-1,1)

In [13]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 3)

<h1>Model Building</h1>

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
regressor = RandomForestRegressor(n_estimators=1000,max_depth=10,random_state=34)

regressor.fit(X_train, np.ravel(Y_train,order='C'))

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=34, verbose=0, warm_start=False)

In [15]:
y_pred = regressor.predict(X_test)

print(r2_score(Y_test,y_pred))

0.8136405572696948


In [16]:
filename = 'Data/resale_model.sav'
pickle.dump(regressor, open(filename, 'wb'))